In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_targets_scored.csv


/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


# **Loading data and see dataset sizes**

In [2]:
df_Train = pd.read_csv("../input/lish-moa/train_features.csv")
df_Test = pd.read_csv("../input/lish-moa/test_features.csv")
#df_Train_target_nonscored = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
df_Train_target_scored = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
sub = pd.read_csv("../input/lish-moa/sample_submission.csv")

target_cols = df_Train_target_scored.columns[1:]


In [3]:
print("training set size = ", df_Train.shape[0])
print("testing set size = ", df_Test.shape[0])

print("train target scored size = ", df_Train_target_scored.shape[0])

training set size =  23814
testing set size =  3982
train target scored size =  23814


# **Information about dataset**

In [4]:
df_Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 876 entries, sig_id to c-99
dtypes: float64(872), int64(1), object(3)
memory usage: 159.2+ MB


In [5]:
df_Train_target_scored.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 207 entries, sig_id to wnt_inhibitor
dtypes: int64(206), object(1)
memory usage: 37.6+ MB


In [6]:
df_Train.describe()

,cp_time,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
count,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,...,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000,23814.000000
mean,48.020156,0.248366,-0.095684,0.152253,0.081971,0.057347,-0.138836,0.035961,-0.202651,-0.190083,...,-0.469244,-0.461411,-0.513256,-0.500142,-0.507093,-0.353726,-0.463485,-0.378241,-0.470252,-0.301505
std,19.402807,1.393399,0.812363,1.035731,0.950012,1.032091,1.179388,0.882395,1.125494,1.749885,...,2.000488,2.042475,2.001714,2.107105,2.159589,1.629291,2.059725,1.703615,1.834828,1.407918
min,24.000000,-5.513000,-5.737000,-9.104000,-5.998000,-6.369000,-10.000000,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000
25%,24.000000,-0.473075,-0.562200,-0.437750,-0.429575,-0.470925,-0.602225,-0.493900,-0.525175,-0.511675,...,-0.566175,-0.565975,-0.589975,-0.568700,-0.563775,-0.567975,-0.552575,-0.561000,-0.592600,-0.562900
50%,48.000000,-0.008850,-0.046600,0.075200,0.008050,-0.026900,-0.015650,-0.000650,-0.017900,0.010000,...,-0.009900,0.003250,-0.009100,-0.013750,-0.003300,-0.010250,-0.001250,-0.006800,0.014000,-0.019500
75%,72.000000,0.525700,0.403075,0.663925,0.463400,0.465375,0.510425,0.528725,0.411900,0.549225,...,0.457750,0.461500,0.445675,0.452900,0.470900,0.444750,0.465225,0.446400,0.461275,0.438650
max,72.000000,10.000000,5.039000,8.257000,10.000000,10.000000,7.282000,7.333000,5.473000,8.887000,...,4.069000,3.960000,3.927000,3.596000,3.747000,2.814000,3.505000,2.924000,3.111000,3.805000


In [7]:
print("Null values in train data: \n", df_Train.isnull().sum())

Null values in train data: 
 sig_id     0
cp_type    0
cp_time    0
cp_dose    0
g-0        0
          ..
c-95       0
c-96       0
c-97       0
c-98       0
c-99       0
Length: 876, dtype: int64


In [8]:
print("Null values in test data: \n", df_Test.isnull().sum())

Null values in test data: 
 sig_id     0
cp_type    0
cp_time    0
cp_dose    0
g-0        0
          ..
c-95       0
c-96       0
c-97       0
c-98       0
c-99       0
Length: 876, dtype: int64


# **Correlation between the columns**

In [9]:
def HighestCorrelation(corrmat):
    c = corrmat.abs()
    shape = c.shape
    s = c.unstack()
    so = s.sort_values(kind="quicksort")
    return so[-(shape[0] + 10):-shape[0]]

In [10]:
corrmat_train = df_Train.corr()
print(HighestCorrelation(corrmat_train))

c-55  c-11    0.914637
c-11  c-55    0.914637
c-33  c-6     0.914730
c-6   c-33    0.914730
c-26  c-13    0.921875
c-13  c-26    0.921875
      c-73    0.923344
c-73  c-13    0.923344
c-42  c-52    0.924619
c-52  c-42    0.924619
dtype: float64


In [11]:
corrmat_train_scored = df_Train_target_scored.corr()
print(HighestCorrelation(corrmat_train_scored))

trpv_agonist                      aldehyde_dehydrogenase_inhibitor    0.528950
aldehyde_dehydrogenase_inhibitor  trpv_agonist                        0.528950
pdgfr_inhibitor                   flt3_inhibitor                      0.705119
flt3_inhibitor                    pdgfr_inhibitor                     0.705119
kit_inhibitor                     flt3_inhibitor                      0.758112
flt3_inhibitor                    kit_inhibitor                       0.758112
pdgfr_inhibitor                   kit_inhibitor                       0.915603
kit_inhibitor                     pdgfr_inhibitor                     0.915603
nfkb_inhibitor                    proteasome_inhibitor                0.921340
proteasome_inhibitor              nfkb_inhibitor                      0.921340
dtype: float64


# **Preprocessing of the data**

In [12]:
def preprocess_df(data):
    data['cp_type'] = (data['cp_type'] == 'trt_cp').astype(int)
    data['cp_dose'] = (data['cp_dose'] == 'D2').astype(int)
    return data

In [13]:
x_train = preprocess_df(df_Train.drop(columns="sig_id"))
x_test = preprocess_df(df_Test.drop(columns="sig_id"))
y_train = df_Train_target_scored.drop(columns="sig_id")
N_FEATURES = x_train.shape[1]
print(x_train.shape)

(23814, 875)


# **Setup hyperparams and useful functions**

In [14]:
SEED = 1235
EPOCHS = 30
BATCH_SIZE = 128
FOLDS = 5
REPEATS = 5
LR = 0.0005
N_TARGETS = len(target_cols)

In [15]:
def multi_log_loss(y_true, y_pred):
    losses = []
    for col in y_true.columns:
        losses.append(log_loss(y_true.loc[:, col], y_pred.loc[:, col]))
    return np.mean(losses)

In [16]:
def seed_everything(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# **Model creation**

In [17]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATURES),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
        
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss="binary_crossentropy", metrics=['accuracy'])
    return model

# **Building training function**

In [18]:
def build_train(resume_models=None, repeat_number=0, folds=5, skip_folds=0):
    
    models = []
    oof_preds = y_train.copy()
    
    kfold = KFold(folds, shuffle=True)
    for fold, (train_ind, val_ind) in enumerate(kfold.split(x_train)):
        print('\n')
        print('-'*50)
        print(f'Training fold {fold + 1}')
        
        cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor="binary_crossentropy", factor=0.4, patience=4, verbose=1, min_delta=0.0001, mode='auto')
        checkpoint_path = f'repeat:{repeat_number}_Folds:{fold}.hdf5'
        cb_checkpt = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor="val_loss", verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        
        model = create_model()
        model.fit(x_train.values[train_ind],
                 y_train.values[train_ind],
                 validation_data=(x_train.values[val_ind], y_train.values[val_ind]),
                 callbacks = [cb_lr_schedule, cb_checkpt],
                 epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2)
        
        model.load_weights(checkpoint_path)
        oof_preds.loc[val_ind, :] = model.predict(x_train.values[val_ind])
        models.append(model)
    
    return models, oof_preds
        

# **execution**

In [19]:
models = []
oof_preds = []

seed_everything(SEED)
for i in range(REPEATS):
    m,oof = build_train(repeat_number= i, folds=FOLDS)
    models = models+m
    oof_preds.append(oof)



--------------------------------------------------
Training fold 1
Epoch 1/30
149/149 - 1s - loss: 0.5646 - accuracy: 0.0383 - val_loss: 0.2169 - val_accuracy: 0.0894
Epoch 2/30
149/149 - 1s - loss: 0.0984 - accuracy: 0.0611 - val_loss: 0.0399 - val_accuracy: 0.0754
Epoch 3/30
149/149 - 1s - loss: 0.0324 - accuracy: 0.0855 - val_loss: 0.0241 - val_accuracy: 0.1048
Epoch 4/30
149/149 - 1s - loss: 0.0235 - accuracy: 0.0956 - val_loss: 0.0210 - val_accuracy: 0.1119
Epoch 5/30
149/149 - 1s - loss: 0.0203 - accuracy: 0.1019 - val_loss: 0.0189 - val_accuracy: 0.0945
Epoch 6/30
149/149 - 1s - loss: 0.0189 - accuracy: 0.1058 - val_loss: 0.0181 - val_accuracy: 0.1144
Epoch 7/30
149/149 - 1s - loss: 0.0180 - accuracy: 0.1148 - val_loss: 0.0176 - val_accuracy: 0.1014
Epoch 8/30
149/149 - 1s - loss: 0.0173 - accuracy: 0.1183 - val_loss: 0.0171 - val_accuracy: 0.1018
Epoch 9/30
149/149 - 1s - loss: 0.0167 - accuracy: 0.1228 - val_loss: 0.0168 - val_accuracy: 0.1067
Epoch 10/30
149/149 - 1s - loss

In [20]:
mean_oof_preds = y_train.copy()
mean_oof_preds.loc[:, target_cols] = 0
for i, p in enumerate(oof_preds):
    print(f"Repeat {i + 1} OOF Log Loss: {multi_log_loss(y_train, p)}")
    mean_oof_preds.loc[:, target_cols] += p[target_cols]
    
mean_oof_preds.loc[:, target_cols] /= len(oof_preds)
print(f"Mean OOF Log loss: {multi_log_loss(y_train, mean_oof_preds)}")
mean_oof_preds.loc[x_train['cp_type'] == 0, target_cols] = 0
print(f"Mean OOF Log Loss(ctl adjusted): {multi_log_loss(y_train, mean_oof_preds)}")


Repeat 1 OOF Log Loss: 0.015310848073686939
Repeat 2 OOF Log Loss: 0.015311869244953431
Repeat 3 OOF Log Loss: 0.01533632537412285
Repeat 4 OOF Log Loss: 0.015363742284964675
Repeat 5 OOF Log Loss: 0.015349812255869155
Mean OOF Log loss: 0.014999690450953335
Mean OOF Log Loss(ctl adjusted): 0.014956785545749047


# **Test prediction and save submission**

In [21]:
test_preds = sub.copy()
test_preds[target_cols] = 0

for model in models:
    test_preds.loc[:, target_cols] += model.predict(x_test)
test_preds.loc[:, target_cols] /= len(models)
test_preds.loc[x_test['cp_type'] == 0, target_cols] = 0
test_preds.to_csv("submission.csv", index=False)